https://github.com/benalexkeen/Introduction-to-linear-programming/

In [1]:
%load_ext nb_black

import pandas as pd

from pulp import LpProblem, LpStatus, LpVariable, lpSum, value
from pulp import LpBinary, LpMinimize, LpMaximize

<IPython.core.display.Javascript object>

In [2]:
factories = pd.read_csv("factory_variables.csv", index_col=["Month", "Factory"])
factories.style.background_gradient("YlGn", subset=["Max_Capacity"])

<IPython.core.display.Javascript object>

In [3]:
demand = pd.DataFrame(
    [
        [1, 120_000],
        [2, 100_000],
        [3, 130_000],
        [4, 130_000],
        [5, 140_000],
        [6, 130_000],
        [7, 150_000],
        [8, 170_000],
        [9, 200_000],
        [10, 190_000],
        [11, 140_000],
        [12, 100_000],
    ],
    columns=["Month", "Demand"],
)
demand = demand.set_index("Month")
demand.style.background_gradient("YlGn", subset=["Demand"])

,Demand
Month,
1,120000
2,100000
3,130000
4,130000
5,140000
6,130000
7,150000
8,170000
9,200000


<IPython.core.display.Javascript object>

In [4]:
def make_logical_and_constraint(y1, x1, x2, target_x1, target_x2):
    """
    Returns a list of constraints for a linear programming model
    that will constrain y1 to 1 when
    x1 = target_x1 and x2 = target_x2; 
    where target_x1 and target_x2 are 1 or 0
    """
    binary = [0, 1]
    assert target_x1 in binary
    assert target_x2 in binary

    if target_x1 == 1 and target_x2 == 1:
        return [y1 >= x1 + x2 - 1, y1 <= x1, y1 <= x2]
    elif target_x1 == 1 and target_x2 == 0:
        return [y1 >= x1 - x2, y1 <= x1, y1 <= (1 - x2)]
    elif target_x1 == 0 and target_x2 == 1:
        return [y1 >= x2 - x1, y1 <= (1 - x1), y1 <= x2]
    else:
        return [y1 >= -(x1 + x2 - 1), y1 <= (1 - x1), y1 <= (1 - x2)]

<IPython.core.display.Javascript object>

In [5]:
prob = LpProblem("Factory_Production_Scheduling", LpMinimize)

<IPython.core.display.Javascript object>

In [6]:
# Production
production = LpVariable.dicts(
    "production",
    ((month, factory) for month, factory in factories.index),
    lowBound=0,
    cat="Integer",
)

# Factory Status, On or Off
factory_status = LpVariable.dicts(
    "factory_status",
    ((month, factory) for month, factory in factories.index),
    cat="Binary",
)

# Factory switch on or off
switch_on = LpVariable.dicts(
    "switch_on", ((month, factory) for month, factory in factories.index), cat="Binary"
)

<IPython.core.display.Javascript object>

In [7]:
# Select index on factory A or B
factory_A_index = [idx for idx in factories.index if idx[1] == "A"]
factory_B_index = [idx for idx in factories.index if idx[1] == "B"]

# Define objective function
prob += lpSum(
    [
        production[m, f] * factories.loc[(m, f), "Variable_Costs"]
        for m, f in factories.index
    ]
    + [
        factory_status[m, f] * factories.loc[(m, f), "Fixed_Costs"]
        for m, f in factories.index
    ]
    + [switch_on[m, f] * 20_000 for m, f in factory_A_index]
    + [switch_on[m, f] * 400_000 for m, f in factory_B_index]
)

<IPython.core.display.Javascript object>

In [8]:
# Production in any month must be equal to demand
months = demand.index
for month in months:
    prob += (
        production[(month, "A")] + production[(month, "B")]
        == demand.loc[month, "Demand"]
    )

# Production in any month must be between minimum and maximum capacity, or zero.
for month, factory in factories.index:
    min_production = factories.loc[(month, factory), "Min_Capacity"]
    max_production = factories.loc[(month, factory), "Max_Capacity"]
    prob += (
        production[(month, factory)] >= min_production * factory_status[month, factory]
    )
    prob += (
        production[(month, factory)] <= max_production * factory_status[month, factory]
    )

# Factory B is off in May
prob += factory_status[5, "B"] == 0
prob += production[5, "B"] == 0

<IPython.core.display.Javascript object>

In [9]:
for month, factory in factories.index:
    # In month 1, if the factory ison, we assume it turned on
    if month == 1:
        prob += switch_on[month, factory] == factory_status[month, factory]

    # In other months, if the factory is on in the current month AND off in the previous month, switch on = 1
    else:
        for constraint in make_logical_and_constraint(
            switch_on[month, factory],
            factory_status[month, factory],
            factory_status[month - 1, factory],
            0,
            1,
        ):
            prob += constraint

<IPython.core.display.Javascript object>

In [10]:
prob.solve()

LpStatus[prob.status]

'Optimal'

<IPython.core.display.Javascript object>

In [11]:
output = []
for month, factory in production:
    var_output = {
        "Month": month,
        "Factory": factory,
        "Production": production[(month, factory)].varValue,
        "Factory Status": factory_status[(month, factory)].varValue,
        "Switch On": switch_on[(month, factory)].varValue,
    }
    output.append(var_output)
output_df = pd.DataFrame.from_records(output).sort_values(["Month", "Factory"])
output_df.set_index(["Month", "Factory"], inplace=True)
output_df

Production  Factory Status  Switch On
Month Factory                                       
1     A           70000.0             1.0        1.0
      B           50000.0             1.0        1.0
2     A           45000.0             1.0        0.0
      B           55000.0             1.0        0.0
3     A           70000.0             1.0        0.0
      B           60000.0             1.0        0.0
4     A           30000.0             1.0        0.0
      B          100000.0             1.0        0.0
5     A          140000.0             1.0        0.0
      B               0.0             0.0        1.0
6     A           60000.0             1.0        0.0
      B           70000.0             1.0        0.0
7     A           90000.0             1.0        0.0
      B           60000.0             1.0        0.0
8     A           70000.0             1.0        0.0
      B          100000.0             1.0        0.0
9     A          100000.0             1.0        0.0
      B          100000.0             1.0        0.0
10    A          170000.0             1.0        0.0
      B           20000.0             1.0        0.0
11    A           80000.0             1.0        0.0
      B           60000.0             1.0        0.0
12    A           80000.0             1.0        0.0
      B           20000.0             1.0        0.0

<IPython.core.display.Javascript object>

In [12]:
value(prob.objective)

13827600.0

<IPython.core.display.Javascript object>